# Day 3: Gear Ratios

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

## Part 1

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


### Code

In [70]:
%reload_ext ipython_unittest

class Solver:
    def __init__(self):
        self.foundNumbers = []
        self.symbols = []
        self.possibleGears = []
    
    def parse( self, document ):
        documentLines = document.split( '\n' )            
        y = 0
        numberBuffer = ""
        for line in documentLines:
            symbolLine = []                       
            x = 0                                  
            for char in line:
                isCharSymbol = False                
                if char.isnumeric( ):
                    numberBuffer += char                    
                else:
                    if numberBuffer != "":
                        self.foundNumbers.append( { 'num': int( numberBuffer ), 'x': x - len(numberBuffer), 'y':y, 'length': len( numberBuffer ) } )
                        numberBuffer = ""
                    if char != '.':
                        isCharSymbol = True
                        if char == '*':
                            self.possibleGears.append( { 'x': x, 'y':y } )
                symbolLine.append( isCharSymbol )    
                x += 1
            self.symbols.append( symbolLine )
            if numberBuffer != "":
                self.foundNumbers.append( { 'num': int( numberBuffer ), 'x': x- len(numberBuffer), 'y':y, 'length': len( numberBuffer ) } )
                numberBuffer = ""            
            y += 1
        #print( self.symbols )
        
    def getHitRangeOfNumber(self, foundNumber ):
        return { 'yStart': max( 0, foundNumber['y']-1 ),
                 'xStart': max( 0, foundNumber['x']-1 ) ,       
                 'yEnd' : min( len( self.symbols )-1, foundNumber['y']+1 ),
                 'xEnd' : min( len( self.symbols[ 0 ] )-1, foundNumber['x'] + foundNumber[ 'length'] ) }

    def isPartNumber( self, foundNumber ):        
        hitRange = self.getHitRangeOfNumber( foundNumber )
        #print(foundNumber)
        #print( {'yStart':yStart, 'yEnd':yEnd, 'xStart':xStart, 'xEnd':xEnd } )
        for y in range( hitRange['yStart'], hitRange['yEnd']+1 ):
            for x in range ( hitRange['xStart'], hitRange['xEnd']+1 ):
                #print( { 'y':y, 'x':x, 'symbol':self.symbols[y][x] } )
                if self.symbols[y][x] == True:
                    #print( 'Found part number '+ str( foundNumber['num'] ) )
                    return True
        # print( 'Not a part number '+ str( foundNumber['num'] ) )
        return False
        

    def getPart1Solution(self):
        solution = 0
        for foundNumber in self.foundNumbers:
            if self.isPartNumber( foundNumber ): solution += foundNumber[ 'num' ]
        return solution
    
    def isPartAttachedToGear( self, part, gear ):
        hitRange = self.getHitRangeOfNumber( part )
        if gear['x'] >= hitRange['xStart'] and gear['x'] <= hitRange['xEnd'] and gear['y'] >= hitRange['yStart'] and gear['y'] <= hitRange['yEnd']:
               return True
        return False
    
    def getAdjacentPartsOfGear( self, gear ):
        parts = []
        for foundNumber in self.foundNumbers:
            if self.isPartNumber( foundNumber ):
                if self.isPartAttachedToGear( foundNumber, gear ):
                    parts.append( foundNumber['num'] )
        return parts
    
    def getPart2Solution(self):
        solution = 0
        for possibleGear in self.possibleGears:
            adjacentParts = self.getAdjacentPartsOfGear( possibleGear )
            if len(adjacentParts) == 2:
                solution += adjacentParts[0] * adjacentParts[1]            
        return solution
    
f = open("day3_input.txt", 'r') 
document = f.read()
solver = Solver( )
solver.parse( document )

### Unit Tests

In [71]:
%%unittest_main

sampleInput = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

class MyTest(unittest.TestCase):
    def setUp(self):
        self.cut = Solver(  )
        
    def test_getPart1SolutionForSample(self):        
        self.cut.parse( sampleInput  )
        self.assertEqual(self.cut.getPart1Solution(), 4361)

    

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Solution

In [72]:
print( solver.getPart1Solution(  ) )

529618


## Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

### Unit Tests Part 2

In [73]:
%%unittest_testcase
def setUp(self):
    self.cut = Solver(  )

def test_getPart2SolutionForSample(self):        
    self.cut.parse( sampleInput  )
    self.assertEqual(self.cut.getPart2Solution( ), 467835)

Success

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Solution Part 2

In [74]:
print( solver.getPart2Solution( ) )

77509019
